In [66]:
# Python
import pandas as pd
import numpy as np
#
from prophet import Prophet
import matplotlib.pyplot as plt
import plotly.graph_objects as go
#
# Python
from prophet.serialize import model_to_json, model_from_json
#
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

#
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [67]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

### Train & Plot Methods

In [68]:
def train(dataframe, months=12):
    df_train_long = dataframe.reset_index().melt(id_vars=['Date'], var_name='ticker', value_name='y')
    df_train_long.rename(columns={'Date': 'ds'}, inplace=True)
    
    # model
    models = {}
    forecasts = {}
    
    for ticker, data in df_train_long.groupby('ticker'):
        model = Prophet()
        model.fit(data[['ds', 'y']])  # Train model
    
        future = model.make_future_dataframe(periods=months, freq='ME')  # Forecast next 12 months
        forecast = model.predict(future)
    
        models[ticker] = model
        forecasts[ticker] = forecast
        
    return forecasts

def plot(dataframe, forecasts, months=12):
    # Allocate the last 5 years of data for testing
    min_date = pd.to_datetime(dataframe.index[-1]).replace(day=1) - pd.DateOffset(months=months)
    min_datestr = min_date.strftime('%Y-%m-%d')
    
    # max_date = min_date + pd.DateOffset(months=1)
    # max_datestr = max_date.strftime('%Y-%m-%d')

    df_train = dataframe.loc[dataframe.index < min_datestr]
    # df_test = dataframe.loc[dataframe.index >= min_datestr]

    # Collect 'ds' (date) and 'yhat' from each forecast
    forecast_dfs = [item[['ds', 'yhat']].rename(columns={'yhat': stock}) for stock, item in forecasts.items()]

    # Merge all forecasts on 'ds' (date)
    merged_forecast = forecast_dfs[0]
    for df in forecast_dfs[1:]:
        merged_forecast = merged_forecast.merge(df, on='ds', how='outer')

    # Compute the mean 'yhat' per time point
    y_pred = merged_forecast.iloc[:, 1:].mean(axis=1)
    y_true = dataframe.mean(axis=1)

    #
    train_true_list = y_pred[:len(df_train)]
    test_true_list = y_pred[len(df_train):]

    # Create the plot
    fig = go.Figure()

    # Add the timeseries line
    fig.add_trace(go.Scatter(y=y_true, x=dataframe.index.tolist(), mode='lines', name='Actual returns',
                             line=dict(color='#5c839f', width=2)))  #, line=dict(color='red'))
    # Add the training plot in red
    fig.add_trace(go.Scatter(y=train_true_list, x=dataframe.index.tolist()[:len(train_true_list)],
                             mode='lines', name='Train returns',
                             line=dict(color='red', width=2)))  #, line=dict(color='red')

    # Add the testing plot in green
    fig.add_trace(go.Scatter(y=test_true_list, x=dataframe.index.tolist()[len(train_true_list):],
                             mode='lines', name='Test returns',
                             line=dict(color='green', width=2)))  # , line=dict(color='green')

    fig.add_vline(x=min_datestr, line_color='red', line_dash='dash', line_width=1)

    # Update layout with labels
    fig.update_layout(
        title='1 Year Prediction vs Actual Plot',
        xaxis=dict(
            title='Date'
        ),
        yaxis=dict(
            title='Day closing return (%)',
            tickformat='.0%',
            range=[0.75, 1.6]
        ),
        legend=dict(title="Legend"),
        template="plotly_white"
    )
    
    fig.show()

### 1 Month Actual vs Prediction

In [69]:
forecasts_1m = train(dataframe=df, months=1)

In [70]:
plot(dataframe=df, forecasts=forecasts_1m, months=1)


### 6 Months Actual vs Prediction

In [71]:
forecasts_6m = train(dataframe=df, months=6)

In [72]:
plot(dataframe=df, forecasts=forecasts_6m, months=6)


### 12 months Actual vs Prediction

In [73]:
forecasts_12m = train(dataframe=df, months=12)

In [74]:
plot(dataframe=df, forecasts=forecasts_12m, months=12)

## Sharpe Ratio

In [94]:
def build_efficient_frontier(forecasts):
    # Create DataFrame of forecasted prices
    # Collect 'ds' (date) and 'yhat' from each forecast
    forecast_dfs = [item[['ds', 'yhat']].rename(columns={'yhat': stock}) for stock, item in forecasts.items()]
    
    # Merge all forecasts on 'ds' (date)
    merged_forecast = forecast_dfs[0]
    for df in forecast_dfs[1:]:
        merged_forecast = merged_forecast.merge(df, on='ds', how='outer')
    merged_forecast
    
    merged_forecast = merged_forecast.set_index('ds')

    # Calculate expected returns and sample covariance
    mu_0 = expected_returns.mean_historical_return(merged_forecast)
    
    # Get only tickers with a mean historical return of at least 5% 
    optimal_tickers = mu_0[mu_0 > 0.05].index
    df_optimal = merged_forecast[optimal_tickers]
    
    mu = expected_returns.mean_historical_return(df_optimal)
    S = risk_models.CovarianceShrinkage(df_optimal).ledoit_wolf() # risk_models.sample_cov, # Ledoit-Wolf shrinkage (df_optimal, frequency=12), # Exponential Covariance
    
    
    # Optimize for maximal Sharpe ratio
    ef = EfficientFrontier(mu, S)
    ef_new = EfficientFrontier(mu, S)
    
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    ef.save_weights_to_file("weights.csv")  # saves to file
    #
    ef.portfolio_performance(verbose=True)
    
    return (df_optimal, raw_weights)

### 1 Month

In [95]:
df_forecasts_1m, raw_weights_1m = build_efficient_frontier(forecasts_1m)

Expected annual return: 14.1%
Annual volatility: 73.2%
Sharpe Ratio: 0.16


### 6 Months

In [96]:
df_forecasts_6m, raw_weights_6m = build_efficient_frontier(forecasts_6m)

Expected annual return: 13.3%
Annual volatility: 131.8%
Sharpe Ratio: 0.09


### 12 Months

In [97]:
df_forecasts_12m, raw_weights_12m = build_efficient_frontier(forecasts_12m)

Expected annual return: 41.4%
Annual volatility: 1108.4%
Sharpe Ratio: 0.04


## Optimal Allocation

### 1 Month

In [98]:
latest_prices = get_latest_prices(df_forecasts_1m)

da = DiscreteAllocation(raw_weights_1m, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: €{:.2f}".format(leftover))

Discrete allocation: {'7564.T': 155, 'RWS.L': 127, 'ILM1.DE': 118, 'BIRD': 117, 'IVT': 121, '6951.T': 77, 'RENT': 76, 'GME': 62, 'AMC': 63, 'NEM.DE': 60, 'TWKS': 60, 'BRKR': 56, 'RCH.L': 55, 'EVC': 50, 'HOV': 53, 'NEO': 49, 'MHO': 46, 'MAN': 46, 'POWW': 50, 'BSL.DE': 49, 'SSP': 40, 'CENT': 47, 'BKNG': 45, 'TPX': 44, 'CACI': 43, '6702.T': 44, 'ALV.DE': 43, 'NBPE.L': 44, 'NYT': 42, 'ONT.L': 42, 'KELYA': 40, 'ITI': 43, 'SMCI': 42, 'CLMB': 40, 'VRTX': 41, 'NTGR': 38, 'IMAX': 37, '7944.T': 43, 'AFL': 39, 'STAA': 39, 'MBG.DE': 38, 'CAR': 38, 'ONL': 39, 'KFY': 36, 'BGFV': 36, '8897.T': 38, '2327.T': 38, 'SAP.DE': 36, '2767.T': 40, 'VSEC': 35, 'STVG.L': 39, 'DXLG': 38, '7936.T': 36, 'HSTM': 34, '8803.T': 36, 'STX': 33, '6588.T': 35, 'WOOF': 36, 'ZUMZ': 35, 'SUP': 35, '9697.T': 35, 'LGEN.L': 33, 'NSP': 32, 'DRH': 31, 'GBF.DE': 31, 'NL': 30, 'AGYS': 31, 'AV.L': 32, '4751.T': 31, 'ASGN': 31, 'MUV2.DE': 31, 'MRL.L': 33, 'MOD': 31, '9107.T': 30, 'SMDS.L': 29, '6736.T': 32, 'FOSL': 31, 'NFG.L': 29, 

### 6 Months

In [99]:
latest_prices = get_latest_prices(df_forecasts_6m)

da = DiscreteAllocation(raw_weights_6m, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: €{:.2f}".format(leftover))

Discrete allocation: {'CENT': 227, 'GME': 173, 'RGL.L': 146, 'DXLG': 134, 'ILM1.DE': 157, 'AMC': 126, 'AGS': 121, 'NXST': 112, 'PMTS': 107, 'CAR': 100, 'JSG.L': 103, 'RENT': 103, '7944.T': 103, 'IMAX': 85, 'WIX': 82, 'SIG': 82, 'ARLO': 84, 'FNKO': 87, '7936.T': 84, 'WOOF': 84, 'NEM.DE': 83, '7564.T': 84, 'OSW': 77, 'NEO': 78, 'ZUMZ': 81, 'TLYS': 77, 'PFC.L': 78, 'CRCT': 77, 'AGYS': 68, 'BSL.DE': 73, 'HWG.L': 67, 'CLMB': 67, '7419.T': 67, 'PETQ': 67, '7846.T': 69, 'GT': 63, 'EFC': 64, '7832.T': 64, 'SREI.L': 60, 'ENPH': 63, 'FOUR.L': 65, 'GLOB': 57, 'AG1.DE': 59, 'SUP': 60, 'RARE': 58, '7906.T': 57, 'ANET': 56, 'IVAC': 49, '6702.T': 56, 'CNK': 55, 'FLTR.L': 52, 'PINE.L': 55, 'DHC': 53, 'TPL': 53, 'BBW': 51, '6951.T': 49, 'PSTG': 52, 'BKNG': 52, 'FGP.L': 50, '6915.T': 49, 'TWKS': 52, 'GBF.DE': 49, '6425.T': 48, 'AMS.L': 49, 'ISRG': 46, 'ICFI': 47, 'DFIN': 48, 'COUR': 48, '4641.T': 46, '8014.T': 48, 'BKR': 45, 'MRL.L': 48, 'OLED': 41, '2767.T': 49, 'RELL': 44, 'SCVL': 44, 'AIV': 45, 'BIG'

### 12 Months

In [100]:
latest_prices = get_latest_prices(df_forecasts_12m)

da = DiscreteAllocation(raw_weights_12m, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: €{:.2f}".format(leftover))

Discrete allocation: {'BGEO.L': 763, 'ILM1.DE': 387, 'BIRD': 250, '8766.T': 42, 'GROW.L': 171, 'CRCT': 144, 'HOV': 137, '3668.T': 134, 'ARLO': 121, 'AG1.DE': 117, '8057.T': 112, 'TWKS': 115, 'IVT': 111, 'FUBO': 104, 'SOHO.L': 94, '2767.T': 99, 'GME': 86, 'FOUR.L': 93, 'ONTF': 88, 'RENT': 94, '8309.T': 86, '8308.T': 86, 'JSG.L': 88, '7936.T': 80, 'BBW': 78, 'COUR': 76, 'PMTS': 78, '7239.T': 75, 'SSP': 68, 'CNK': 75, '8111.T': 73, 'JBL': 69, 'BCPT.L': 66, 'MRVI': 68, 'LQDT': 65, '7740.T': 65, '3104.T': 65, 'CMLS': 61, 'ITI': 65, 'GCI': 64, 'SMCI': 63, 'ONL': 62, '3738.T': 60, 'STEM.L': 56, '4326.T': 54, 'ONT.L': 56, 'LRN': 55, 'TYMN.L': 52, '9766.T': 53, 'BCYC': 52, 'ENPH': 52, 'AHT.L': 51, 'NXT.L': 51, 'TTWO': 50, '7564.T': 53, 'SUP': 54, '7508.T': 52, 'NWG.L': 50, 'SHYF': 50, 'AGS': 50, 'NBPE.L': 49, '6745.T': 48, '9076.T': 48, 'QUAD': 48, 'SDRY.L': 44, '7912.T': 47, 'RWI.L': 45, 'FOSL': 48, 'AGR.L': 46, '8966.T': 45, 'SNWS.L': 44, 'NEXN.L': 43, '8953.T': 42, 'BEZ.L': 42, 'FNKO': 44, '